In [ ]:
"""
긍정 기사 상위 1048576개, 부정 기사 상위 1048576개를 각각 하나의 파일로 생성한다. 
new_dict를 생성하고 후에는 BERT fine-tuning을 위한 파일이다. 
상위 기사를 따로 추출하는 이유는, 확실히 긍정 또는 부정인 기사를 사용하기 위해서이다. 
전체 성별 기사는 10784446이고 한 엑셀의 최대 row는 1048576로, 
전체 성별 기사의 10%인 1078444와 유사하므로 최대 row를 기준으로 상위 파일을 선별한다.

알고리즘은 간단하다. 
1048576개까지 append하다가, 그 이후부터는 append될 때 마다 
최소 또는 최대 score를 하나씩 제거하여 1048576개를 유지한다. 
"""

import os
import pandas as pd
import time

start = time.time()

version = "knu_score" #"new_dict_score"
top = 1048576
qlist = ["여성", "여자+-여성", "남성", '남자+-남성']

pos_score, neg_score = list(), list()
df_pos = pd.DataFrame(columns=['title', 'source', 'date', 'text', version])
df_neg = pd.DataFrame(columns=['title', 'source', 'date', 'text', version])
df_pos_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', '#3_'+version, f"{version+'_pos'}.xlsx")
df_neg_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', '#3_'+version, f"{version+'_neg'}.xlsx")

for query in qlist:
    for year in range(2000, 2020):
        for month in range(1, 13):
            xlsx_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', '#3_'+version, "#3_"+query, 
                                     f"{'#3_'+str(year)+'_'+str(month).zfill(2)+'_'+query}.xlsx")

            df = pd.read_excel(xlsx_name, index_col=0)
            df = df.drop(columns=['url'])
            for i in range(len(df)):

                d = df.loc[i]
                k = d['knu_score']

                if k > 0:
                    pos_score.append(k)
                    df_pos.append(d)
                if len(pos_score) > top:
                    ind = pos_score.index(min(pos_score))
                    pos_score.pop(ind)
                    df_pos.drop(ind)
                    df_pos.reset_index(drop=True)

                if k < 0:
                    neg_score.append(k)
                    df_neg.append(d)
                if len(neg_score) > top:
                    ind = neg_score.index(max(neg_score))
                    neg_score.pop(ind)
                    df_neg.drop(ind)
                    df_neg.reset_index(drop=True)
                            
df_pos.to_excel(df_pos_name)
df_neg.to_excel(df_neg_name)

end = time.time()
print(end - start)